###  Machine Learning e Data Science com Python de A à Z (Classificacão) - IA Expert Academy

In [ ]:
# Instalação

#!pip -q install plotly
#!pip -q install yellowbrick


In [ ]:
# Importando as bibliotecas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Base de dados de crédito

- Fonte (adaptado): https://www.kaggle.com/laotse/credit-risk-dataset

### Exploração dos Dados

In [ ]:
df_credit = pd.read_csv('credit-data.csv')

df_credit.head() # Exibindo as primeiras linhas do dataset

#### Vamos analisar os atributos da base de dados de crédito e definir o tipo de variável de cada um deles.

- **Atributos:**
    - **clientid** - Id do cliente -> Variável categórica nominal
    - **Income** - Renda -> Variável numérica contínua
    - **Age** - Idade -> Variável numérica contínua
    - **Loan** - Dívida -> Variável numérica contínua
    - **c#default** - Pagou/ñ pagou emprestimo -> Variável numérica discreta -> **Variável target**
    
obs. default = 0 -> Pagou

obs. default = 1 -> Não pagou

#### Objetivo:

- Nosso objetivo é criar um modelo de Machine Learning para prever se um cliente irá pagar ou não o empréstimo de acordo com as variáveis presentes acima.

In [ ]:
df_credit.tail() # Exibindo as últimas linhas do dataset

In [ ]:
df_credit.describe() # Exibindo informações estatísticas do dataset

In [ ]:
df_credit[df_credit['income'] >= 69995.685578] # Exibindo os registros da pessoa com maior renda

### Visualização dos Dados

In [ ]:
# Exibindo a quantidade de registros para cada classe da variável default
np.unique(df_credit['default'], return_counts=True) 

#Resultado :

# 0: 1717 registros -> pagantes
# 1: 283 registros -> não pagantes
# 1° análise : A base de dados está desbalanceada, pois a quantidade de pagantes é muito maior que a quantidade de não pagantes.

In [ ]:
sns.countplot(x='default', data=df_credit);
plt.title('Quantidade de pagantes e não pagantes');

In [ ]:
plt.hist(df_credit['age'], bins=20, color='blue', edgecolor='black');
plt.title('Distribuição da idade');

In [ ]:
plt.hist(df_credit['income'], color='blue', edgecolor='black');
plt.title('Distribuição da renda');

In [ ]:
plt.hist(df_credit['loan'], color='blue', edgecolor='black');
plt.title('Distribuição do empréstimo');

In [ ]:
grafico = px.scatter_matrix(df_credit, dimensions=['age', 'income' ,'loan'], color='default')
grafico.show()
#scatter => gráfico de dispersão interessante para observar valores que estão fora do quadrante de normalidade

Após a análise pelo gráfico de dispersão, conseguimos encontrar alguns padrões de usuários não pagantes e também 3 anomalias na coluna 'Age' com idades negativas que iremos corrigir agora

### Tratamentos dos valores inconsistentes

In [ ]:
df_credit.loc[df_credit['age'] < 0]

#### Técnicas para tratar valores negativos

In [ ]:
# Apagar a coluna inteira (não recomendado quando se tem poucos dados inconsistentes)
df_credit2 = df_credit.drop('age' , axis=1)
df_credit2.head()

In [ ]:
# Apagar somente os registros com valores inconsistentes
indices = df_credit[df_credit['age'] < 0].index
df_credit3 = df_credit.drop(indices)
df_credit3

In [ ]:
# Validando se ainda existem valores de idade menores que 0
df_credit3.loc[df_credit3['age'] < 0]

In [ ]:
# Preencher os valores inconsistentes manualmente (Recomendado quando se tem poucos dados inconsistentes)

# Aqui iremos preencher o valor de idade negativo com a média das idades

df_credit['age'].mean()

# Nota-se que aqui a um problema, pois essa média calculada tem como unidade de medida os valores inconsistentes, 
#ou seja, a média está incorreta.

In [ ]:
# Achando a média real

df_credit['age'][df_credit['age'] > 0].mean() # Média é igual a 40.92

In [ ]:
# Preenchendo os valores inconsistentes com a média correta

df_credit.loc[df_credit['age'] < 0, 'age'] = 40.92

# Validando se ainda existem valores de idade menores que 0

df_credit.loc[df_credit['age'] < 0]

df_credit.head(27)

### Tratamento de Valores Faltantes

In [ ]:
df_credit.isnull().sum() # Verificando se existem valores nulos

df_credit.loc[pd.isnull(df_credit['age'])]

In [ ]:
# Preencher os registros nulos com a média

df_credit['age'].fillna(df_credit['age'].mean() , inplace=True)

In [ ]:
# Validando se os Valores Nulos foram registrados com a Média

df_credit.loc[(df_credit['clientid']== 29) | (df_credit['clientid']== 30) | (df_credit['clientid']== 31)]

### Divisão da base de dados em treino e teste (Previsores e Classe)

Normalmente chama-se a base de dados de previsores de X e a base de dados de classe de y.

In [ ]:
# Iremos pegar todos os registros das colunas "income" , "age" e "loan"
x_credit = df_credit.iloc[ : , 1:4].values
x_credit # Exibindo os valores, nota-se que foi convertido os valores de dataframe do pandas para array do numpy
         # Isto foi feito com o ".values" para que converta pois os modelos de ML só conseguem trabalhar com arrays do numpy

In [107]:
# Pegando os valores da coluna "default" que é a nossa variável alvo

y_credit = df_credit.iloc[ : , 4].values
y_credit

array([0, 0, 0, ..., 1, 0, 0])

### Escalonamento dos Valores

In [108]:
x_credit

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [113]:
x_credit[:,0].min() , x_credit[:,1].min() , x_credit[:,2].min()

(np.float64(20014.4894700497),
 np.float64(18.055188510566897),
 np.float64(1.37762959325451))

In [114]:
x_credit[: , 0].max() , x_credit[: , 1].max() , x_credit[: , 2].max()

(np.float64(69995.6855783239),
 np.float64(63.971795841120205),
 np.float64(13766.0512393337))

Comparando os valores vimos que algumas variáveis são muito maiores que outras e vice-versa , isso pode fazer com que na hora que iremos aplicar algoritmos de Machine learning encima desses dados ele possa se enviesar e achar que variáveis com valores maiores são mais importantes do que variáveis com valores menores, para isso vamos aplicar a técnica de escalonamento de variáveis.

In [115]:
from sklearn.preprocessing import StandardScaler # Importando a biblioteca para normalização dos dados

scaler_credit = StandardScaler()
x_credit = scaler_credit.fit_transform(x_credit)

In [117]:
x_credit[:,0].min() , x_credit[:,1].min() , x_credit[:,2].min()

(np.float64(-1.7676158019964077),
 np.float64(-1.7264145408889917),
 np.float64(-1.4592791099462408))

In [118]:
x_credit[: , 0].max() , x_credit[: , 1].max() , x_credit[: , 2].max()

(np.float64(1.7220222385319197),
 np.float64(1.7393673928651967),
 np.float64(3.0616609141708273))

Nota-se que agora os dados estão mais escalonados e os algoritmso de Machine Learning irão se comportar melhor.

## Base de Dados do Censo